In [0]:
# !pip install hoge
# ドライブをマウント
from google.colab import drive
drive.mount('/content/gdrive')
# カレントディレクトリの変更
import os
os.chdir('/content/gdrive/My Drive/NAL-LAB/Colab Notebooks/')
!pwd

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/NAL-LAB/Colab Notebooks


In [0]:
# google colab で chainer などを使うための設定

# colab の cuda に応じて、いい感じに chainer と Cupy をインストールするコマンド
!curl https://colab.chainer.org/install | sh -

# chainer のインストール確認コマンド
!python -c "import chainer; chainer.print_runtime_info()"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   3062      0 --:--:-- --:--:-- --:--:--  3056
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
+ pip install -q cupy-cuda100  chainer 
+ set +ex
Installation succeeded!
Platform: Linux-4.14.79+-x86_64-with-Ubuntu-18.04-bionic
Chainer: 5.0.0
NumPy: 1.14.6
CuPy:
  CuPy Version          : 5.2.0
  CUDA Root             : /usr/local/cuda
  CUDA Build Version    : 10000
  CUDA Driver Version   : 10000
  CUDA Runtime Version  : 10000
  cuDNN Build Version   : 7301
  cuDNN Version         : 7301
  NCCL Build Version    : 2307
iDeep: 2.0.0.post3


In [0]:
import re
import numpy as np
import chainer
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chainer import serializers

In [0]:
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

GPU availability: True
cuDNN availablility: True


In [0]:
# モデルの保存
serializers.save_npz("sentence_classifier_lstm.npz", model) # npz形式で書き出し

In [0]:
# モデルのロード
gpu = 0

# 定数
epoch_num = 10
embed_size = 200
hidden_size = 100
batch_size = 5
out_size = 2

# モデルの定義
model = L.Classifier(SentenceClassifierLSTM(
    vocab_size=len(words),
    embed_size=embed_size,
    hidden_size=hidden_size,
    out_size=out_size
))

# モデルのロード
serializers.load_npz("sentence_classifier_lstm.npz", model)

NameError: ignored

In [0]:
# ファイル読み込み
## リストへ格納
with open("vector.txt") as f:
    type(f)
    vector = [s.strip() for s in f.readlines()]

In [0]:
# 上記とは別に取り込む floatとして
import numpy as np
vector_list = np.loadtxt('vector.txt', delimiter=',', dtype='float')

In [0]:
tag_list = np.loadtxt('tag.txt', delimiter='\n', dtype='int32')

In [0]:
# データ分割
import pandas as pd
from sklearn.model_selection import train_test_split
# 結果をシードで指定
data_train, data_test, labels_train, labels_test = train_test_split(vector_list, tag_list, train_size=0.80,random_state=1)
# リストの結合2
vectors2 = list(zip(data_train,labels_train))
# dataへの代入
data = vectors2

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [0]:
with open("tag.txt") as f:
    tag_list = [s.strip() for s in f.readlines()]
    print(tag)

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '1', '0', '0', '1', '1', '1', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0']


In [0]:
# リストの結合
#vectors = list(zip(vector,tags_list))

In [0]:
# リストの結合2
vectors = list(zip(vector_list,tag_list))

In [0]:
# dataへの代入

data = vectors

In [0]:
class SentenceClassifierLSTM(chainer.Chain):
    
    def __init__(self, vocab_size, embed_size, hidden_size, out_size):
        """クラスの初期化
        
        Args:
            vocab_size: 単語数
            embed_size: 埋め込みベクトルサイズ
            hidden_size: 隠れ層サイズ
            out_size: 出力層サイズ
        """
        super(SentenceClassifierLSTM, self).__init__(
            # encode用のLink関数
            xe = L.EmbedID(vocab_size, embed_size, ignore_label=-1),
            eh = L.LSTM(embed_size, hidden_size),
            hh = L.Linear(hidden_size, hidden_size),
            # classifierのLink関数
            hy = L.Linear(hidden_size, out_size)
        )
        
    def __call__(self, x):
        """順伝播の計算を行う関数
        
        Args:
            x:　入力値
        Retruns:
            y: 
        """
        # エンコード
        x = F.transpose_sequence(x)
        self.eh.reset_state()
        for word in x:
            e = self.xe(word)
            h = self.eh(e)            
        # 分類
        y = self.hy(h)
        return y

In [0]:
N = len(data)
data_x, data_t = [], []
for d in data:
    data_x.append(d[0]) # 文書
    data_t.append(d[1]) # ラベル

def sentence2words(sentence):# 文章の前処理
    stopwords = ["i", "a", "an", "the", "and", "or", "if", "is", "are", "am", "it", "this", "that", "of", "from", "in", "on"]
    #sentence = sentence.lower() # 小文字化
    #sentence = sentence.replace("\n", "") # 改行削除
    #sentence = re.sub(re.compile(r"[!-\/:-@[-`{-~]"), " ", sentence) # 記号をスペースに置き換え
    #sentence = sentence.split(" ") # スペースで区切る
    sentence_words = []
    for word in sentence:
        #if (re.compile(r"^.*[0-9]+.*$").fullmatch(word) is not None): # 数字が含まれるものは除外
        #    continue
        if word in stopwords: # ストップワードに含まれるものは除外
            continue
        sentence_words.append(word)        
    return sentence_words # 

# 単語辞書 # 
words = {}
for sentence in data_x:
    sentence_words = sentence2words(sentence)
    for word in sentence_words:
        if word not in words:
            words[word] = len(words)
            
# 文章を単語ID配列にする
data_x_vec = []
for sentence in data_x:
    sentence_words = sentence2words(sentence)# 
    sentence_ids = []
    for word in sentence_words:
        sentence_ids.append(words[word])
    data_x_vec.append(sentence_ids)
    
# 文章の長さを揃えるため、-1パディングする（系列を覚えておきやすくするため、前パディングする）
max_sentence_size = 0
for sentence_vec in data_x_vec:
    if max_sentence_size < len(sentence_vec):
        max_sentence_size = len(sentence_vec)
for sentence_ids in data_x_vec:
    while len(sentence_ids) < max_sentence_size:
        sentence_ids.insert(0, -1) # 先頭に追加
    
# データセット
data_x_vec = np.array(data_x_vec, dtype="int32")# ここfloat
data_t = np.array(data_t, dtype="int32")
dataset = []
for x, t in zip(data_x_vec, data_t):
    dataset.append((x, t)) 

In [0]:
gpu = 0

# 定数
epoch_num = 20
embed_size = 200
hidden_size = 100
batch_size = 5
out_size = 2

# モデルの定義
model = L.Classifier(SentenceClassifierLSTM(
    vocab_size=len(words),
    embed_size=embed_size,
    hidden_size=hidden_size,
    out_size=out_size
))
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

if gpu >= 0:
    
    chainer.cuda.get_device(gpu).use()
    model.to_gpu(gpu)

In [0]:
# 学習
# # 学習

# split_at = int(len(dataset) * 0.8)
# train, test = chainer.datasets.split_dataset(dataset, split_at)


train, test = chainer.datasets.split_dataset_random(dataset, N-20)
train_iter = chainer.iterators.SerialIterator(train, batch_size)
test_iter = chainer.iterators.SerialIterator(test, batch_size, repeat=False, shuffle=False)
updater = chainer.training.StandardUpdater(train_iter, optimizer, device=gpu)
trainer = chainer.training.Trainer(updater, (epoch_num, "epoch"), out="result")
trainer.extend(extensions.Evaluator(test_iter, model, device=gpu))
trainer.extend(extensions.LogReport(trigger=(1, "epoch")))
trainer.extend(extensions.PrintReport( ["epoch", "main/loss", "validation/main/loss", "main/accuracy", "validation/main/accuracy", "elapsed_time"]))
#trainer.extend(extensions.ProgressBar()) # プログレスバー出力
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
1           0.691186    0.71784               0.5            0.45                      15.3691       
2           0.376685    0.714619              0.95           0.5                       20.4946       
3           0.228143    0.711945              1              0.5                       24.6607       
4           0.138408    0.70858               1              0.5                       29.7806       
5           0.0971347   0.706399              1              0.5                       33.9659       
6           0.0604543   0.703988              1              0.5                       39.1007       
7           0.0404894   0.701838              1              0.5                       44.213        
8           0.0300963   0.700609              1              0.5                       48.397        
9           0.0235756   0.699227              1              0.55                   

In [0]:
d[0]

In [0]:
d[1]

In [0]:
print(data_x_vec[0])

In [0]:
data_t

In [0]:
dataset

In [0]:
# tfの方
import numpy as np
tf_vector_list = np.loadtxt('tf.txt', delimiter=',', dtype='float32')
tag_list = np.loadtxt('tag.txt', delimiter='\n', dtype='int32')

In [0]:
# tf-ifの方
import numpy as np
tf_idf_vector_list = np.loadtxt('tf-idf.txt', delimiter=',', dtype='float')
tag_list = np.loadtxt('tag.txt', delimiter='\n', dtype='int32')

In [0]:
# リストの結合
#vectors = list(zip(tf_vector_list,tag_list))
vectors = list(zip(tf_idf_vector_list,tag_list))

In [0]:
vectors

In [0]:
# arrayではないのかも
# ファイル読み込み
## リストへ格納
with open("tf-idf.txt") as f:
    tf_idf_vector_list = [s.strip() for s in f.readlines()] # これだと、e+が残る

In [0]:
with open("tag.txt") as f:
    tag_list = [s.strip() for s in f.readlines()]

In [0]:
# リストの結合
vectors = list(zip(tf_vector_list,tag_list))
data = vectors

In [0]:
# リストの結合
vectors = list(zip(tf_idf_vector_list,tag_list))
data = vectors

# 混同行列
confusion_matrix()は、公式ドキュメントによると、第一引数が真値で、第二引数が識別器による判別値になっているようなので、それぞれそのように渡す。

In [0]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [0]:
print(test_iter)

In [0]:
len(vector_list[0])

300